# Model creation

In [1]:
%%bash
pip install wandb

In [1]:
import json
import pprint

import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.utils import class_weight
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

2024-03-31 23:56:10.951919: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-31 23:56:10.975043: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Dataset

In [2]:
df = pd.read_csv(
    "drug_consumption_2.txt",
    header=None,
    names=[
        "idx",
        "N-Score",
        "E-Score",
        "O-Score",
        "A-Score",
        "C-Score",
        "Impulsiveness",
        "Sensation-seeking",
        "Amphet",
        "Benzo",
        "Cannabis",
        "Heroin",
        "Ketamine",
        "Methadone",
        "Semeron",
    ],
)

df.head()

,idx,N-Score,E-Score,O-Score,A-Score,C-Score,Impulsiveness,Sensation-seeking,Amphet,Benzo,Cannabis,Heroin,Ketamine,Methadone,Semeron
0,1,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,0,0,0,0,0,0,0
1,2,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,0,0,0,0,0,0,0
2,3,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,0,0,0,0,0,0,0
3,4,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,0,0,0,0,0,0,0
4,5,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,0,0,0,0,0,0,0


In [3]:
# Normalize score ranges to be [0, 1].
for column in df.columns[1:8]:
    column_min, column_max = df[column].min(), df[column].max()

    column_normalized = (df[column] - column_min) / (column_max - column_min)

    df[column] = column_normalized

In [4]:
print("Column ranges:")
for column in df.columns:
    column_range = (df[column].min(), df[column].max())

    print(f"   { column}: [{column_range[0]}, { column_range[1] }]")

Column ranges:
   idx: [1, 1888]
   N-Score: [0.0, 1.0]
   E-Score: [0.0, 1.0]
   O-Score: [0.0, 1.0]
   A-Score: [0.0, 1.0]
   C-Score: [0.0, 1.0]
   Impulsiveness: [0.0, 1.0]
   Sensation-seeking: [0.0, 1.0]
   Amphet: [0, 1]
   Benzo: [0, 1]
   Cannabis: [0, 1]
   Heroin: [0, 1]
   Ketamine: [0, 1]
   Methadone: [0, 1]
   Semeron: [0, 1]


In [5]:
print("Class counts:")

for target in df.iloc[:, 8:15].columns:
    # Get train and test data splits, stratisfy for target.
    target_train_df, target_test_df = train_test_split(
        df, train_size=0.8, shuffle=True, stratify=df[target], random_state=0
    )

    # Get input and target from the data split.
    target_x_train, target_y_train = (
        target_train_df.iloc[:, 1:8],
        target_train_df.iloc[:, 8:15][target],
    )
    target_x_test, target_y_test = (
        target_test_df.iloc[:, 1:8],
        target_test_df.iloc[:, 8:15][target],
    )

    print(
        f"  { target } dataset\n     Train dataset: 0: { len(target_y_train[target_y_train == 0]) }, 1: { len(target_y_train[target_y_train == 1]) }\n     Test dataset: 0: { len(target_y_test[target_y_test == 0]) }, 1: { len(target_y_test[target_y_test == 1]) }"
    )

Class counts:
  Amphet dataset
     Train dataset: 0: 1426, 1: 82
     Test dataset: 0: 357, 1: 20
  Benzo dataset
     Train dataset: 0: 1432, 1: 76
     Test dataset: 0: 358, 1: 19
  Cannabis dataset
     Train dataset: 0: 1138, 1: 370
     Test dataset: 0: 284, 1: 93
  Heroin dataset
     Train dataset: 0: 1485, 1: 23
     Test dataset: 0: 371, 1: 6
  Ketamine dataset
     Train dataset: 0: 1505, 1: 3
     Test dataset: 0: 376, 1: 1
  Methadone dataset
     Train dataset: 0: 1450, 1: 58
     Test dataset: 0: 362, 1: 15
  Semeron dataset
     Train dataset: 0: 1502, 1: 6
     Test dataset: 0: 375, 1: 2


## Decision Tree

In [6]:
models = {}
metrics = {}

for target in df.iloc[:, 8:15].columns:
    # Get train and test data splits, stratisfy for target.
    target_train_df, target_test_df = train_test_split(
        df, train_size=0.8, shuffle=True, stratify=df[target], random_state=0
    )

    # Get input and target from the data split.
    target_x_train, target_y_train = (
        target_train_df.iloc[:, 1:8],
        target_train_df.iloc[:, 8:15][target],
    )
    target_x_test, target_y_test = (
        target_test_df.iloc[:, 1:8],
        target_test_df.iloc[:, 8:15][target],
    )

    # Create classifier.
    target_clf = DecisionTreeClassifier(
        criterion="gini",
        max_depth=15,
        min_samples_leaf=3,
        class_weight="balanced",
        random_state=0,
    )

    # Train model with data specified for target.
    target_clf.fit(target_x_train, target_y_train)
    models[target] = target_clf

    joblib.dump(
        target_clf, f"./models/decision_tree_{ target.lower() }.joblib", compress=3
    )

    # Evaluate trained classifier.
    target_y_predictions = target_clf.predict(target_x_test)

    # Calculate metrics.
    accuracy = accuracy_score(target_y_test, target_y_predictions)
    precision = precision_score(
        target_y_test, target_y_predictions, zero_division=np.nan
    )
    recall = recall_score(target_y_test, target_y_predictions)
    f1 = f1_score(target_y_test, target_y_predictions)

    metrics[target] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
    }

pprint.pprint(models)
print(json.dumps(metrics, indent=4))

{'Amphet': DecisionTreeClassifier(class_weight='balanced', max_depth=15,
                       min_samples_leaf=3, random_state=0),
 'Benzo': DecisionTreeClassifier(class_weight='balanced', max_depth=15,
                       min_samples_leaf=3, random_state=0),
 'Cannabis': DecisionTreeClassifier(class_weight='balanced', max_depth=15,
                       min_samples_leaf=3, random_state=0),
 'Heroin': DecisionTreeClassifier(class_weight='balanced', max_depth=15,
                       min_samples_leaf=3, random_state=0),
 'Ketamine': DecisionTreeClassifier(class_weight='balanced', max_depth=15,
                       min_samples_leaf=3, random_state=0),
 'Methadone': DecisionTreeClassifier(class_weight='balanced', max_depth=15,
                       min_samples_leaf=3, random_state=0),
 'Semeron': DecisionTreeClassifier(class_weight='balanced', max_depth=15,
                       min_samples_leaf=3, random_state=0)}
{
    "Amphet": {
        "accuracy": 0.8620689655172413,
    

## K-NN

In [7]:
models = {}
metrics = {}

for target in df.iloc[:, 8:15].columns:
    # Get train and test data splits, stratisfy for target.
    target_train_df, target_test_df = train_test_split(
        df, train_size=0.8, shuffle=True, stratify=df[target], random_state=0
    )

    # Get input and target from the data split.
    target_x_train, target_y_train = (
        target_train_df.iloc[:, 1:8],
        target_train_df.iloc[:, 8:15][target],
    )
    target_x_test, target_y_test = (
        target_test_df.iloc[:, 1:8],
        target_test_df.iloc[:, 8:15][target],
    )

    # Create classifier.
    target_neigh = KNeighborsClassifier(
        n_neighbors=3,
    )

    # Train model with data specified for target.
    target_neigh.fit(target_x_train, target_y_train)
    models[target] = target_neigh

    joblib.dump(target_neigh, f"./models/knn_{ target.lower() }.joblib", compress=3)

    # Evaluate trained classifier.
    target_y_predictions = models[target].predict(target_x_test)

    # Calculate metrics.
    accuracy = accuracy_score(target_y_test, target_y_predictions)
    precision = precision_score(
        target_y_test, target_y_predictions, zero_division=np.nan
    )
    recall = recall_score(target_y_test, target_y_predictions)
    f1 = f1_score(target_y_test, target_y_predictions)

    metrics[target] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
    }

pprint.pprint(models)
print(json.dumps(metrics, indent=4))

{'Amphet': KNeighborsClassifier(n_neighbors=3),
 'Benzo': KNeighborsClassifier(n_neighbors=3),
 'Cannabis': KNeighborsClassifier(n_neighbors=3),
 'Heroin': KNeighborsClassifier(n_neighbors=3),
 'Ketamine': KNeighborsClassifier(n_neighbors=3),
 'Methadone': KNeighborsClassifier(n_neighbors=3),
 'Semeron': KNeighborsClassifier(n_neighbors=3)}
{
    "Amphet": {
        "accuracy": 0.9442970822281167,
        "precision": 0.3333333333333333,
        "recall": 0.05,
        "f1_score": 0.08695652173913043
    },
    "Benzo": {
        "accuracy": 0.9283819628647215,
        "precision": 0.1,
        "recall": 0.05263157894736842,
        "f1_score": 0.06896551724137931
    },
    "Cannabis": {
        "accuracy": 0.7002652519893899,
        "precision": 0.375,
        "recall": 0.3225806451612903,
        "f1_score": 0.3468208092485549
    },
    "Heroin": {
        "accuracy": 0.9840848806366048,
        "precision": NaN,
        "recall": 0.0,
        "f1_score": 0.0
    },
    "Ketamine"

## Random Forest

In [8]:
models = {}
metrics = {}

for target in df.iloc[:, 8:15].columns:
    # Get train and test data splits, stratisfy for target.
    target_train_df, target_test_df = train_test_split(
        df, train_size=0.8, shuffle=True, stratify=df[target], random_state=0
    )

    # Get input and target from the data split.
    target_x_train, target_y_train = (
        target_train_df.iloc[:, 1:8],
        target_train_df.iloc[:, 8:15][target],
    )
    target_x_test, target_y_test = (
        target_test_df.iloc[:, 1:8],
        target_test_df.iloc[:, 8:15][target],
    )

    # Create classifier.
    target_clf = RandomForestClassifier(
        n_estimators=50,
        max_depth=25,
        min_samples_leaf=3,
        class_weight="balanced",
        random_state=0,
    )

    # Train model with data specified for target.
    target_clf.fit(target_x_train, target_y_train)
    models[target] = target_clf

    joblib.dump(
        target_clf, f"./models/random_forest_{ target.lower() }.joblib", compress=3
    )

    # Evaluate trained classifier.
    target_y_predictions = target_clf.predict(target_x_test)

    # Calculate metrics.
    accuracy = accuracy_score(target_y_test, target_y_predictions)
    precision = precision_score(
        target_y_test, target_y_predictions, zero_division=np.nan
    )
    recall = recall_score(target_y_test, target_y_predictions)
    f1 = f1_score(target_y_test, target_y_predictions)

    metrics[target] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
    }

pprint.pprint(models)
print(json.dumps(metrics, indent=4))

{'Amphet': RandomForestClassifier(class_weight='balanced', max_depth=25,
                       min_samples_leaf=3, n_estimators=50, random_state=0),
 'Benzo': RandomForestClassifier(class_weight='balanced', max_depth=25,
                       min_samples_leaf=3, n_estimators=50, random_state=0),
 'Cannabis': RandomForestClassifier(class_weight='balanced', max_depth=25,
                       min_samples_leaf=3, n_estimators=50, random_state=0),
 'Heroin': RandomForestClassifier(class_weight='balanced', max_depth=25,
                       min_samples_leaf=3, n_estimators=50, random_state=0),
 'Ketamine': RandomForestClassifier(class_weight='balanced', max_depth=25,
                       min_samples_leaf=3, n_estimators=50, random_state=0),
 'Methadone': RandomForestClassifier(class_weight='balanced', max_depth=25,
                       min_samples_leaf=3, n_estimators=50, random_state=0),
 'Semeron': RandomForestClassifier(class_weight='balanced', max_depth=25,
                    

## ANN

In [20]:
def get_data_from_df(data_frame: pd.DataFrame, target: str):
    # Get train and test data splits, stratisfy for target.
    target_train_df, target_test_df = train_test_split(
        data_frame,
        train_size=0.8,
        shuffle=True,
        stratify=data_frame[target],
        random_state=0,
    )
    print(target)

    # Get input and target from the data split.
    x_train, y_train = (
        target_train_df.iloc[:, 1:8],
        target_train_df.iloc[:, 8:15][target],
    )
    x_test, y_test = (
        target_test_df.iloc[:, 1:8],
        target_test_df.iloc[:, 8:15][target],
    )

    return x_train, y_train, x_test, y_test

In [10]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bas_korver. Use `wandb login --relogin` to force relogin


True

In [11]:
dataset = wandb.Artifact("drug_consumption", type="dataset")
dataset.add_file("./drug_consumption_2.txt")
with wandb.init(project="XAI-group-assignment", job_type="upload"):
    wandb.log_artifact(dataset)

In [12]:
def create_ann_model_sweep():
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Dense(
                wandb.config.layer_1,
                activation=wandb.config.activation_1,
                input_shape=wandb.config.input_shape_1,
            ),
            tf.keras.layers.Dense(
                wandb.config.layer_2,
                activation=wandb.config.activation_2,
            ),
            tf.keras.layers.Dense(
                wandb.config.layer_3,
                activation=wandb.config.activation_3,
            ),
        ]
    )

    match wandb.config.optimizer:
        case "adam":
            optimizer = tf.keras.optimizers.Adam
        case "adamw":
            optimizer = tf.keras.optimizers.AdamW
        case "adamax":
            optimizer = tf.keras.optimizers.Adamax
        case "sgd":
            optimizer = tf.keras.optimizers.SGD

    model.compile(
        optimizer=optimizer(wandb.config.lr),
        loss=wandb.config.loss,
        metrics=["accuracy", "precision", "recall"],
    )

    return model

In [15]:
def train_with_wandb():
    # Initialize wandb
    with wandb.init():
        # Load data.
        target_x_train, target_y_train, target_x_test, target_y_test = get_data_from_df(
            df, wandb.config.drug
        )

        # Calculate class weights.
        if wandb.config.class_weighting:
            target_class_weights = dict(
                enumerate(
                    class_weight.compute_class_weight(
                        "balanced",
                        classes=np.unique(target_y_train),
                        y=target_y_train,
                    )
                )
            )

        # Create classifier.
        target_model = create_ann_model_sweep()

        # Train model with data specified for target.
        if wandb.config.class_weighting:
            target_model.fit(
                target_x_train,
                target_y_train,
                epochs=wandb.config.epoch,
                batch_size=wandb.config.batch_size,
                class_weight=target_class_weights,
                callbacks=[WandbMetricsLogger("batch")],
            )
        else:
            target_model.fit(
                target_x_train,
                target_y_train,
                epochs=wandb.config.epoch,
                batch_size=wandb.config.batch_size,
                callbacks=[WandbMetricsLogger("batch")],
            )

        models[target] = target_model

        target_model.save(f"./models/ann_{ target }.h5")

        # Evaluate trained classifier.
        target_y_predictions = (target_model.predict(target_x_test) >= 0.5).astype(
            "int32"
        )

        # Calculate metrics.
        accuracy = accuracy_score(target_y_test, target_y_predictions)
        precision = precision_score(target_y_test, target_y_predictions)
        recall = recall_score(target_y_test, target_y_predictions)
        f1 = f1_score(target_y_test, target_y_predictions)

        wandb.log(
            {
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
                "f1_score": f1,
            }
        )

        model_artifact = wandb.Artifact(
            name=f"{target}_{f1}",
            type="model",
            description=f"Model trained for {target}",
            metadata=dict(wandb.config),
        )
        model_artifact.add_file(f"./models/ann_{ target }.h5")
        wandb.log_artifact(model_artifact)

In [16]:
for drug in df.iloc[:, 8:15].columns:
    sweep_configuration = {
        "name": f"sweep_{drug}",
        "method": "bayes",
        "metric": {
            "goal": "maximize",
            "name": "f1_score",
        },
        "parameters": {
            "optimizer": {
                "distribution": "categorical",
                "values": [
                    "adam",
                    "adamw",
                    "adamax",
                    "sgd",
                ],
            },
            "batch_size": {
                "distribution": "q_uniform",
                "q": 8,
                "min": 8,
                "max": 256,
            },
            "epoch": {
                "distribution": "int_uniform",
                "min": 3,
                "max": 100,
            },
            "lr": {
                "distribution": "uniform",
                "min": 0.00001,
                "max": 0.1,
            },
            "class_weighting": {
                "distribution": "categorical",
                "values": [
                    True,
                    False,
                ],
            },
            "layer_1": {
                "distribution": "constant",
                "value": 64,
            },
            "activation_1": {
                "distribution": "constant",
                "value": "relu",
            },
            "input_shape_1": {
                "distribution": "constant",
                "value": (7,),
            },
            "layer_2": {
                "distribution": "constant",
                "value": 32,
            },
            "activation_2": {
                "distribution": "constant",
                "value": "relu",
            },
            "layer_3": {
                "distribution": "constant",
                "value": 1,
            },
            "activation_3": {
                "distribution": "constant",
                "value": "sigmoid",
            },
            "loss": {
                "distribution": "constant",
                "value": "binary_crossentropy",
            },
            "drug": {
                "distribution": "constant",
                "value": drug,
            },
        },
    }

    # Setup sweep
    sweep_id = wandb.sweep(sweep=sweep_configuration, project="XAI-group-assignment")
    wandb.agent(sweep_id=sweep_id, function=train_with_wandb, count=500)

    # Stop sweep recording.
    wandb.teardown()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 9kg7b063
Sweep URL: https://wandb.ai/bas_korver/XAI-group-assignment/sweeps/9kg7b063


wandb: Agent Starting Run: as2ydkcv with config:
wandb: 	activation_1: relu
wandb: 	activation_2: relu
wandb: 	activation_3: sigmoid
wandb: 	batch_size: 248
wandb: 	class_weighting: False
wandb: 	drug: Amphet
wandb: 	epoch: 30
wandb: 	input_shape_1: [7]
wandb: 	layer_1: 64
wandb: 	layer_2: 32
wandb: 	layer_3: 1
wandb: 	loss: binary_crossentropy
wandb: 	lr: 0.08002929024687723
wandb: 	optimizer: adamw
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6102 - loss: 0.7660 - precision: 0.0453 - recall: 0.3195        
Epoch 2/45
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1530 - loss: 0.6833 - precision: 0.0563 - recall: 0.9588 
Epoch 3/45
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6993 - loss: 0.6857 - precision: 0.1031 - recall: 0.5116 
Epoch 4/45
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3513 - loss: 0.6866 - precision: 0.0662 - recall: 0.8029 
Epoch 5/45
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4320 - loss: 0.6615 - precision: 0.0701 - recall: 0.7953 
Epoch 6/45
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6670 - loss: 0.6309 - precision: 0.0774 - recall: 0.5483 
Epoch 7/45
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5409 - loss: 0.6527 - precision: 0.0778 - recall: 0.6861 
Epoch 8/45
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7902 - loss: 0.6389 - precision: 0.0696 - recall: 0.2526 
Epoch 9/45
19/19 ━━━━━━━━━━

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


wandb: Ctrl + C detected. Stopping sweep.


KeyboardInterrupt: 

In [26]:
def create_ann_model():
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Dense(
                wandb.config.layer_1,
                activation=wandb.config.activation_1,
                input_shape=wandb.config.input_shape_1,
            ),
            tf.keras.layers.Dense(
                wandb.config.layer_2, activation=wandb.config.activation_2
            ),
            tf.keras.layers.Dense(
                wandb.config.layer_3, activation=wandb.config.activation_3
            ),
        ]
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=wandb.config.loss,
        metrics=wandb.config.metric,
    )

    return model

In [ ]:
def initialize_wandb(target):
    wandb.init(
        # set the wandb project where this run will be logged
        project="XAI-group-assignment",
        name=f"{target}",
        group="run_epoch_25_batch_16_lr_0.001(default)",
        # track hyperparameters and run metadata with wandb.config
        config={
            "layer_1": 64,
            "activation_1": "relu",
            "input_shape_1": (7,),
            "layer_2": 32,
            "activation_2": "relu",
            "layer_3": 1,
            "activation_3": "sigmoid",
            "optimizer": "adam",
            "loss": "binary_crossentropy",
            "metric": ["accuracy", "precision", "recall"],
            "epoch": 25,
            "batch_size": 16,
        },
    )
    # return wandb.config

In [28]:
models = {}
metrics = {}

for target in df.iloc[:, 8:15].columns:
    target_x_train, target_y_train, target_x_test, target_y_test = get_data_from_df(
        df, target
    )

    # Calculate class weights.
    target_class_weights = dict(
        enumerate(
            class_weight.compute_class_weight(
                "balanced", classes=np.unique(target_y_train), y=target_y_train
            )
        )
    )

    # Initialize wandb
    initialize_wandb(target)

    # Create classifier.
    target_model = create_ann_model()

    # Train model with data specified for target.
    target_model.fit(
        target_x_train,
        target_y_train,
        epochs=wandb.config.epoch,
        batch_size=wandb.config.batch_size,
        class_weight=target_class_weights,
        verbose=0,
        callbacks=[
            WandbMetricsLogger(),
        ],
    )
    models[target] = target_model

    target_model.save(f"./models/ann_{ target }.h5")

    # Evaluate trained classifier.
    target_y_predictions = (target_model.predict(target_x_test) >= 0.5).astype("int32")

    # Calculate metrics.
    accuracy = accuracy_score(target_y_test, target_y_predictions)
    precision = precision_score(target_y_test, target_y_predictions)
    recall = recall_score(target_y_test, target_y_predictions)
    f1 = f1_score(target_y_test, target_y_predictions)

    metrics[target] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
    }
    wandb.log(
        {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
        }
    )

    model_artifact = wandb.Artifact(
        name=f"{target}_{f1}",
        type="model",
        description=f"Model trained for {target}",
        metadata=dict(wandb.config),
    )
    wandb.add_file(f"ann_{ target }.h5")
    wandb.log_artifact(model_artifact)

    wandb.finish()

pprint.pprint(models)
print(json.dumps(metrics, indent=4))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


accuracy,▁
epoch/accuracy,█▁▁▄▃▂▂▃▁▃▂▃▂▃▂▃▃▂▄▃▃▃▄▃▃
epoch/epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
epoch/loss,█▇▅▅▄▄▄▃▅▄▃▃▄▃▃▃▃▂▂▂▂▂▁▁▂
epoch/precision,▄▁▂▅▇▄▅▆▂▅▄▆▃▆▅▆▆▄▇▆▆▇█▆▅
epoch/recall,▁▇█▆▇▇▇▇▇▇▇▇▇▇█▇▇█▇▇▇█▇█▇
f1_score,▁
precision,▁
recall,▁
accuracy,0.59416
epoch/accuracy,0.6061


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


...

accuracy,▁
epoch/accuracy,▁▆▅▇█▄▇▆▇█▆██▇▇▇▇█▇▇█▆▇█▇
epoch/epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
epoch/loss,█▇▇▆▇▄▄▄▃▃▃▄▃▄▃▃▂▃▂▃▃▃▁▂▁
epoch/precision,▁▃▄▅▆▅▆▅▅█▅█▆▆▆▅▆▇▇▅▇▆▇▆▆
epoch/recall,▆▁▄▂▂█▄▅▂▄▇▅▃▄▄▄▅▄▇▃▄▇▅▂▄
f1_score,▁
precision,▁
recall,▁
accuracy,0.57825
epoch/accuracy,0.6565


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


accuracy,▁
epoch/accuracy,▆█▆▅▆▁▆▄▄▂▃▄▃▂▅▅▃▃▃▃▄▄▅▄▃
epoch/epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
epoch/loss,█▅▆▅▅▄▇▄▄▃▃▄▃▃▃▄▂▂▃▂▂▁▂▂▁
epoch/precision,▄▁▆▄▇▅▁▆▆▆▆▇▅▅█▆▆▆▄▇▆██▇▇
epoch/recall,▂▁▄▄▅█▁▇▇██▇▇█▇▅▇█▅█▇█▇▇█
f1_score,▁
precision,▁
recall,▁
accuracy,0.56499
epoch/accuracy,0.5179


{'Amphet': <Sequential name=sequential_14, built=True>,
 'Benzo': <Sequential name=sequential_15, built=True>,
 'Cannabis': <Sequential name=sequential_16, built=True>,
 'Heroin': <Sequential name=sequential_17, built=True>,
 'Ketamine': <Sequential name=sequential_18, built=True>,
 'Methadone': <Sequential name=sequential_19, built=True>,
 'Semeron': <Sequential name=sequential_20, built=True>}
{
    "Amphet": {
        "accuracy": 0.5941644562334217,
        "precision": 0.10179640718562874,
        "recall": 0.85,
        "f1_score": 0.18181818181818182
    },
    "Benzo": {
        "accuracy": 0.7639257294429708,
        "precision": 0.1276595744680851,
        "recall": 0.631578947368421,
        "f1_score": 0.21238938053097345
    },
    "Cannabis": {
        "accuracy": 0.6816976127320955,
        "precision": 0.423728813559322,
        "recall": 0.8064516129032258,
        "f1_score": 0.5555555555555556
    },
    "Heroin": {
        "accuracy": 0.7824933687002652,
        "pre